In [ ]:
# code block to track CPU time and Memory usage
import psutil
import time

start_memory = psutil.virtual_memory().available
start_time = time.time()

In [ ]:
import numpy as np
import pandas as pd
import pickle
import matplotlib as plt
from itertools import combinations
import re

All data and preprocessing results are stored under `.data/`

In [ ]:
data_path='data/' 
exp_id='v0'

Create a code mapping dictionary

In [ ]:
codes={}

Load drug's target (from CTDbase)

In [ ]:
gene_drug=pd.read_csv(data_path+'CTD/drug-gene-CTD_C0000657245_ixns.tsv', sep='\t')

In [ ]:
gene_drug.head()

Save the drug name and MeSH ID mapping for later use

In [ ]:
codes['drugname2mesh']={row[0].upper():row[1] for idx, row in gene_drug[['Chemical Name','Chemical ID']].drop_duplicates().iterrows()}
codes['mesh2drugname']={row[0].upper():row[1] for idx, row in gene_drug[['Chemical ID','Chemical Name']].drop_duplicates().iterrows()}
codes['gene_symbol2id'] = {row[0].upper():row[1] for idx, row in gene_drug[['Gene Symbol','Gene ID']].drop_duplicates().iterrows()}

In [ ]:
gene_drug=gene_drug[['Gene ID', 'Chemical Name']].drop_duplicates()
gene_drug['Chemical Name']=gene_drug['Chemical Name'].apply(lambda x: x.upper() if type(x)==str else x)

gene_drug['Gene ID']=gene_drug['Gene ID'].apply(lambda x: 'gene_'+str(x))
gene_drug['Chemical Name']=gene_drug['Chemical Name'].apply(lambda x: 'drug_'+x)

gene_drug.drop_duplicates(inplace=True)


Load pathways from CTDbase

In [ ]:
pathways=pd.read_csv(data_path+'CTD/pathways-CTD_D000086382_pathways.tsv',sep='\t')
path_sim=pd.concat([pd.DataFrame(list(combinations(pathway,2,)),columns=['gene1','gene2']) for pathway in pathways['Association inferred via'].apply(lambda x: x.split('|') if '|' in x else None).dropna().values]).drop_duplicates()

Load pathways from KEGG. The files were preprocessed as a pairwise gene

In [ ]:
path_sim_kegg=pd.read_csv(data_path+'biology-database/KegglinkevaluationPPPN_1', header=None,sep='\t')
path_sim_kegg.columns=['gene1', 'gene2', 'positive']
path_sim_kegg.replace('PP', 1, inplace=True)
path_sim_kegg.replace('PN', 0, inplace=True)

path_sim_kegg=path_sim_kegg.loc[path_sim_kegg['positive']==1, ['gene1','gene2']]

gene_name =pd.read_excel(data_path+'biology-database/All_Human_Protein_Coding_Genes_3_27_2020.xlsx')
gene_dict= {row['Gene Id']:row['Gene Symbol'] for _, row in gene_name[['Gene Id','Gene Symbol']].iterrows()}

In [ ]:
path_sim_kegg['gene1']=path_sim_kegg['gene1'].apply(lambda x: gene_dict.get(x))
path_sim_kegg['gene2']=path_sim_kegg['gene2'].apply(lambda x: gene_dict.get(x))

In [ ]:
path_sim_kegg.dropna(inplace=True)

Load PHARMAKB [pathways](https://www.pharmgkb.org/page/COVID)

In [ ]:
pathway_ace_inhibitor=list(set(['ATP6AP2', 'MAPK1', 'AGTR2', 'ATP6AP2', 'REN', 'MAS1', 'TGFB1', 'MAPK3', 'ATP6AP2', 'MAPK3',
                       'AGTR1','TGFB1', 'MAPK1', 'NOS3', 'BDKRB2', 'BDKRB2', 'BDKRB1', 'NR3C2', 'CYP11B2', 'AGTR1', 'CYP11B2', 'AGTR1', 'AGT', 'KNG1', 'CYP11B2', 'ACE']))
pathway_fluv=['CYP1A2','CYP2C19','CYP3A']
pathway_losartan=list(set(['AGTR1','CYP2C9',"CYP3A4",'CYP2C9',"CYP3A4",'CYP2C9',"CYP3A4", 'CYP2C9',"CYP3A4", 'UGT1A1',"UGT2B7"]))

In [ ]:
path_sim=pd.concat([path_sim]+[path_sim_kegg]+[pd.DataFrame(list(combinations(pathway,2,)),columns=['gene1','gene2']) for pathway in [pathway_ace_inhibitor,pathway_fluv,pathway_losartan]])

In [ ]:
path_sim['gene1']=path_sim['gene1'].apply(lambda x: codes['gene_symbol2id'].get(x))
path_sim['gene2']=path_sim['gene2'].apply(lambda x: codes['gene_symbol2id'].get(x))
path_sim.dropna(inplace=True)
path_sim['gene1']=path_sim['gene1'].apply(lambda x: 'gene_'+str(int(x)))
path_sim['gene2']=path_sim['gene2'].apply(lambda x: 'gene_'+str(int(x)))
path_sim.drop_duplicates(inplace=True)

In [ ]:
len(path_sim)

Load phenotypes from CTDbase

In [ ]:
phenotypes=pd.read_csv(data_path+'CTD/phenotype-drug-gene-CTD_D000086382_diseases.tsv', sep='\t')

In [ ]:
phenotypes.head()

In [ ]:
codes['phenotype_id_to_name']={row[0]:row[1] for idx, row in phenotypes[['Phenotype Term ID','Phenotype Term Name']].drop_duplicates().iterrows()}

Drugs and phenotypes

In [ ]:
drug_phenotype=phenotypes['Chemical Inference Network'].dropna().apply(lambda x: x.split('|')).apply(pd.Series).merge(phenotypes['Phenotype Term ID'],left_index=True, right_index=True).melt(id_vars=['Phenotype Term ID'],value_name='drug').drop('variable', axis=1).dropna()
drug_phenotype['drug']=drug_phenotype['drug'].apply(lambda x: x.upper())
drug_phenotype.dropna(inplace=True)

drug_phenotype['Phenotype Term ID']=drug_phenotype['Phenotype Term ID'].apply(lambda x: 'phenotype_'+x)
drug_phenotype['drug']=drug_phenotype['drug'].apply(lambda x: 'drug_'+x)
drug_phenotype=drug_phenotype[['drug', 'Phenotype Term ID']]


Genes and phenotypes

In [ ]:
gene_phenotype=phenotypes['Gene Inference Network'].dropna().apply(lambda x: x.split('|')).apply(pd.Series).merge(phenotypes['Phenotype Term ID'],left_index=True, right_index=True).melt(id_vars=['Phenotype Term ID'],value_name='gene').drop('variable', axis=1).dropna()
gene_phenotype['Phenotype Term ID']=gene_phenotype['Phenotype Term ID'].apply(lambda x: 'phenotype_'+x)

gene_phenotype['gene']=gene_phenotype['gene'].apply(lambda x: codes['gene_symbol2id'].get(x))
gene_phenotype.dropna(inplace=True)
gene_phenotype['gene']=gene_phenotype['gene'].apply(lambda x: 'gene_'+str(int(x)))
gene_phenotype=gene_phenotype[['gene', 'Phenotype Term ID']]

In [ ]:
len(set(gene_drug['Chemical Name'].values).intersection(set(drug_phenotype['drug'].values)))

In [ ]:
len(set(drug_phenotype['drug'].values))

Load SARS-CoV-2 baits and host gene interaction from [Gorden et al. Nature 2020](https://www.nature.com/articles/s41586-020-2286-9#Sec36)

In [ ]:
baits_prey=pd.read_csv(data_path+'biology-database/baits-prey-mist.csv')

In [ ]:
baits_prey=baits_prey[['Bait', 'PreyGene']]

In [ ]:
baits_prey['Bait'].nunique(), baits_prey['PreyGene'].nunique()

In [ ]:
baits_prey['Bait']=baits_prey['Bait'].apply(lambda x: 'bait_'+x)
baits_prey['PreyGene']=baits_prey['PreyGene'].apply(lambda x: codes['gene_symbol2id'].get(x))
baits_prey.dropna(inplace=True)
baits_prey['PreyGene']=baits_prey['PreyGene'].apply(lambda x: 'gene_'+str(int(x)))

In [ ]:
#size of drug target, pathway, host gene, phenotype-related genes
gene_drug['Gene ID'].nunique(),len(set(path_sim[['gene1','gene2']].values.ravel())),len(set(baits_prey['PreyGene'].unique())), gene_phenotype['gene'].nunique()

In [ ]:
#number of intersection between host genes and drug target
len(set(baits_prey['PreyGene'].unique()).intersection(gene_drug['Gene ID'].unique()))

In [ ]:
#intersection between target and pathways
len(set(gene_drug['Gene ID'].unique()).intersection(set(path_sim[['gene1','gene2']].values.ravel())))

In [ ]:
#intersection between pathways and host gene
len(set(baits_prey['PreyGene'].unique()).intersection(set(path_sim[['gene1','gene2']].values.ravel())))

In [ ]:
#intersection between pathways, target genes, host gene
len(set(baits_prey['PreyGene'].unique()).intersection(set(path_sim[['gene1','gene2']].values.ravel())).intersection(set(gene_drug['Gene ID'].unique()) ))


Map all the entities (drugs, genes, phenotypes, baits) to ID

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
gene_drug.columns=['node1', 'node2']# gene, drug
path_sim.columns=['node1', 'node2']# gene1, gene2
baits_prey.columns=['node1','node2']#bait, preygene
gene_phenotype.columns=['node1', 'node2'] #gene, phenotype
drug_phenotype.columns=['node1','node2']#drug, phenotye

In [ ]:
gene_drug['type']='gene-drug'
path_sim['type']='gene-gene'
baits_prey['type']='bait-gene'
gene_phenotype['type']='gene-phenotype'
drug_phenotype['type']='drug-phenotype'

edge_index=pd.concat([gene_drug, path_sim, baits_prey, gene_phenotype, drug_phenotype])

edge_index['node1']=edge_index['node1'].astype(str)
edge_index['node2']=edge_index['node2'].astype(str)

Label Encoders

In [ ]:
le=LabelEncoder()
le.fit(np.concatenate((edge_index['node1'], edge_index['node2'])))

In [ ]:
edge_index['node1']=le.transform(edge_index['node1'])
edge_index['node2']=le.transform(edge_index['node2'])

In [ ]:
len(le.classes_)

Obtain pre-trained entity embedding from [DRKG](https://github.com/gnn4dr/DRKG)

In [ ]:
import csv

In [ ]:
#Get pretrained embedding
entity_emb=np.load(data_path+'DRKG/embed/DRKG_TransE_l2_entity.npy')
emb_size=entity_emb.shape[1]

In [ ]:
entity_idmap_file = data_path+'DRKG/embed/entities.tsv'
relation_idmap_file = data_path+'DRKG/embed/relations.tsv'

In [ ]:
baits_drkg=['Disease::'+entity.split('_')[1] for entity  in le.classes_ if entity.split('_')[0]=='bait']
gene_drkg = ['Gene::'+entity.split('_')[1] for entity in le.classes_ if entity.split('_')[0]=='gene']
phenotype_drkg=['Biological Process::'+entity.split('_')[1] for entity in le.classes_ if entity.split('_')[0]=='phenotype']

Map the DRKG's DrugBank ID to MeSH ID

In [ ]:
drugname2external=pd.concat([
    pd.read_csv(data_path+'CTD/drugbank_drugs.csv', encoding = "ISO-8859-1").rename(columns={'drugbank_id':'id'}),
    pd.read_csv(data_path+'CTD/chembl_compound.csv').rename(columns={'name':'drugname'})
]).groupby('drugname', as_index=False).first()

drugname2id={row[0].upper():row[1] for _, row in drugname2external[['drugname', 'id']].iterrows()}
drug_drkg = ['Compound::'+drugname2id.get(entity.split('_')[1],'') for entity in le.classes_ if entity.split('_')[0]=='drug']

Get drugname/disease name to entity ID mappings

In [ ]:
entity_map = {}
entity_id_map = {}
relation_map = {}
with open(entity_idmap_file, newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile, delimiter='\t', fieldnames=['name','id'])
    for row_val in reader:
        entity_map[row_val['name']] = int(row_val['id'])
        entity_id_map[int(row_val['id'])] = row_val['name']
        
with open(relation_idmap_file, newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile, delimiter='\t', fieldnames=['name','id'])
    for row_val in reader:
        relation_map[row_val['name']] = int(row_val['id'])
        
# handle the ID mapping
bait_ids = []
gene_ids = []
drug_ids = []
phenotype_ids = []

    
for bait in baits_drkg:
    bait_ids.append(entity_map.get(bait))

for gene in gene_drkg:
    gene_ids.append(entity_map.get(gene))
    
for drug in drug_drkg:
    drug_ids.append(entity_map.get(drug))
    
for phenotype in phenotype_drkg:
    phenotype_ids.append(entity_map.get(phenotype))


## 1. hybrid embedding: COVID-19 graph + DRKG pre-train

In [ ]:
bait_emb=np.array([entity_emb[bait_id] if bait_id is not None else np.zeros(emb_size) for bait_id in bait_ids ])
drug_emb=np.array([entity_emb[drug_id] if drug_id is not None else np.zeros(emb_size) for drug_id in drug_ids ])
gene_emb=np.array([entity_emb[gene_id] if gene_id is not None else np.zeros(emb_size) for gene_id in gene_ids ])
phenotype_emb=np.array([entity_emb[phenotype_id] if phenotype_id is not None else np.zeros(emb_size) for phenotype_id in phenotype_ids ])


In [ ]:
#How many missing in drugs?
len(drug_ids),len([gene_id for gene_id in drug_ids if gene_id is not None])

In [ ]:
#How many missing in genes?
len(gene_ids),len([gene_id for gene_id in gene_ids if gene_id is not None])

In [ ]:
#How many missing in phenotypes?
len(phenotype_ids),len([gene_id for gene_id in phenotype_ids if gene_id is not None])

The pre-trained embedding is now serived as a node feature

In [ ]:
node_features=np.concatenate((bait_emb, drug_emb, gene_emb, phenotype_emb))

## 2. COVID-19 graph alone embedding

In [ ]:
np.random.rand(4)

In [ ]:
bait_emb_ph = np.random.dirichlet(alpha=np.random.rand(emb_size), size=len(bait_ids))
drug_emb_ph = np.random.dirichlet(alpha=np.random.rand(emb_size), size=len(drug_ids))
gene_emb_ph = np.random.dirichlet(alpha=np.random.rand(emb_size), size=len(gene_ids))
phenotype_emb_ph = np.random.dirichlet(alpha=np.random.rand(emb_size), size=len(phenotype_ids))

In [ ]:
node_features_alone = np.concatenate((bait_emb_ph, drug_emb_ph, gene_emb_ph, phenotype_emb_ph))

In [ ]:
edge_index.to_pickle(data_path+'edge_index_'+exp_id+'.pkl')
pickle.dump(le, open(data_path+'LabelEncoder_'+exp_id+'.pkl','wb'))
pickle.dump(node_features, open(data_path+'node_feature_'+exp_id+'.pkl', 'wb'))
pickle.dump(node_features_alone, open(data_path+'dirichlet_node_feature_'+exp_id+'.pkl', 'wb'))
pickle.dump(codes, open(data_path+'codes_'+exp_id+'.pkl','wb'))

## 3. hybrid embedding without bait-prey information

In [ ]:
edge_index=pd.concat([gene_drug, path_sim, gene_phenotype, drug_phenotype])

edge_index['node1']=edge_index['node1'].astype(str)
edge_index['node2']=edge_index['node2'].astype(str)

In [ ]:
le=LabelEncoder()
le.fit(np.concatenate((edge_index['node1'], edge_index['node2'])))

In [ ]:
edge_index['node1']=le.transform(edge_index['node1'])
edge_index['node2']=le.transform(edge_index['node2'])

In [ ]:
len(le.classes_)

In [ ]:
# handle the ID mapping
gene_ids = []
drug_ids = []
phenotype_ids = []

    
for gene in gene_drkg:
    gene_ids.append(entity_map.get(gene))
    
for drug in drug_drkg:
    drug_ids.append(entity_map.get(drug))
    
for phenotype in phenotype_drkg:
    phenotype_ids.append(entity_map.get(phenotype))


In [ ]:
drug_emb=np.array([entity_emb[drug_id] if drug_id is not None else np.zeros(emb_size) for drug_id in drug_ids ])
gene_emb=np.array([entity_emb[gene_id] if gene_id is not None else np.zeros(emb_size) for gene_id in gene_ids ])
phenotype_emb=np.array([entity_emb[phenotype_id] if phenotype_id is not None else np.zeros(emb_size) for phenotype_id in phenotype_ids ])


In [ ]:
node_features=np.concatenate((bait_emb, drug_emb, gene_emb, phenotype_emb))

In [ ]:
edge_index.to_pickle(data_path+'edge_index_no_bp_'+exp_id+'.pkl')
pickle.dump(le, open(data_path+'LabelEncoder_no_bp_'+exp_id+'.pkl','wb'))
pickle.dump(node_features, open(data_path+'node_feature_no_bp_'+exp_id+'.pkl', 'wb'))

In [ ]:
# code block to track CPU / Memory usage
cpu_sec = time.time() - start_time
end_memory = psutil.virtual_memory().available
mem_use = (start_memory - end_memory) / (1024.0 ** 2)
print(f'CPU time: {cpu_sec:.2f} sec')
print(f'Memory usage: {mem_use:.2f} MB')